# Step 5b: Aggregated Aqueduct data by administrative unit (state and country)

1. Read in basin-state withdrawal data
2. Read in basin Aqueduct indicators data
3. Weight Aqueduct score by polygon's fraction of total admin unit 

In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
# PATHS!
# Country
crtyROOT = r'\Projections\Final_Data\Data\Aqueduct40\step5_country_rankings'
# Final withdrawal data
wwbcPATH = os.path.join(crtyROOT, "withdrawals_basin-states-bias_corrected.csv")  
# Aqueduct indicators
indPATH = r'\Projections\Final_Data\Data\Aqueduct40\step3_calculate_indicators\working'
wsPATH = os.path.join(indPATH, 'Aqueduct40_indicators_annual-exploded-additive.csv')
rootPATH = r'\Projections\Final_Data\Data'
aq3PATH = os.path.join(rootPATH, 'Aqueduct30', 'published', 'aqueduct-30-country-rankings.xlsx')
aq3cartoPATH = os.path.join(rootPATH, 'Aqueduct30', 'published', 'aqueduct_results_v01_{}_v06_CARTO.csv').format
# SAVE LOCATIONS
finalPATH = os.path.join(crtyROOT, 'final', 'y2023m07d05_sk_Aqueduct40_country-state.xlsx')
cartoPATH = os.path.join(rootPATH, 'Aqueduct40', 'step4_final_data_download', 'carto', 'aqueduct_results_{}_2023.csv').format
workingPATH = os.path.join(crtyROOT, "Aqueduct40_rankings_{}_working.csv").format                   
                         

scenFolders = ['ssp126',
               'ssp370',
               'ssp585']
gcmFolders = ['gfdl-esm4',
              'ipsl-cm6a-lr',
              'mpi-esm1-2-hr',
              'mri-esm2-0',
              'ukesm1-0-ll']

ws_labels = {
#     -5 : 'Zero Supply',
    -9999 : 'No Data',
    -1: 'Arid and Low Water Use',
    0: 'Low (<10%)',
    1: 'Low - Medium (10-20%)',
    2: 'Medium - High (20-40%)',
    3: 'High (40-80%)', 
    4: 'Extremely High (>80%)'
}
wd_labels = {
#     -5 : 'Zero Supply',
    -9999 : 'No Data',
    -1: 'Arid and Low Water Use',
    0: 'Low (<5%)',
    1: 'Low - Medium (5-25%)',
    2: 'Medium - High (25-50%)',
    3: 'High (50-75%)', 
    4: 'Extremely High (>75%)'
}

iv_labels = {
#     -5 : 'Zero Supply',
    -9999 : 'No Data',
    0: 'Low (<0.25)',
    1: 'Low - Medium (0.25-0.50)',
    2: 'Medium - High (0.50-0.75)',
    3: 'High (0.75-1.00)', 
    4: 'Extremely High (>1.00)'
}

ind_names = {'ws_s': 'bws', 'wd_s': 'bwd', 'iv_s': 'iav'}

# Function to create category value from score for all indicators
def category(score, df_in):
    cat = 'cat'
    df_cat = np.floor(df_in[score]).to_frame(name = cat)
    df_cat[cat] = np.where(df_cat[cat] == 5, 4, df_cat[cat])
    return df_cat

# 1. Read in final withdrawal data

The values represent million cubic meters of annual demand per sector per milestone year. Irrigation data is the only sector that is calculated from PCR-GLOBWB model ouputs (because its's a function of climate). Demand and industry are projected by SSP, and are the same across the 5 GCMs per scenarion. Livestock withdrawal data ends in 2019. All future values equal the 2019 value. 



In [3]:
# Read in final demand data
df_ww = pd.read_csv(wwbcPATH, index_col = ['basin_stat', 'year'])
df_wwm = df_ww.melt(ignore_index = False)
df_wwm[['sector', 'gcm', 'ssp']] = df_wwm['variable'].str.split('_',  expand=True)
df_wwm['scen'] = df_wwm['gcm']  + "_" + df_wwm['ssp']
# Clean data. 
# -- Remove 2014 rows
df_wwm.reset_index(inplace = True)
df_wwm = df_wwm.loc[df_wwm['year'] > 2014]
# -- Rename baseline 
df_wwm.loc[df_wwm['scen'] == 'gswp3-w5e5_historical', 'scen'] = 'baseline_hist'
# -- Drop null rows based on year-ssp combo
df_wwm = df_wwm.loc[~((df_wwm.year.isin([2030, 2050, 2080])) & (df_wwm.ssp == 'historical'))]
df_wwm =  df_wwm.loc[~((df_wwm.year.isin([2019])) & (df_wwm.ssp != 'historical'))]
# -- Add IDs for catchcments and admins
df_wwm[['pfaf_id', 'gid_1']] = df_wwm['basin_stat'].str.split('-',  expand=True)
df_wwm['gid_0'] = df_wwm['gid_1'].apply(lambda x: x[0:3])
# Set values as integers
df_wwm['year'] = df_wwm['year'].astype(int)
df_wwm['pfaf_id'] = df_wwm['pfaf_id'].astype(int)

# 2. Read in Aqueduct 4.0 indicator data

This data represents 1 step before the "published" data. It has a water stress score for each GCM/scenario, rather than the median. Data is in its "exploded" form, meaning each row is a unique combo of catchment, year, scen, and gcm, while the columns are Aqueduct values like raw, score, cat, label per indicator

In [4]:
# Read in Aqueduct indicator data
df_ws = pd.read_csv(wsPATH, index_col = ['pfaf_id', 'period', 'scen'])
# Keep water stress, water depletion, and interannual
indicators = ['ws_s', 'wd_s', 'iv_s']
df_wsf = df_ws.filter(indicators).reset_index()
# Clean index. Make sure things are integers, and the naming is consistent with withdrawal data
df_wsf.rename(columns = {'period': 'year'}, inplace = True)
df_wsf['year'] = df_wsf['year'].astype(int)
df_wsf['pfaf_id'] = df_wsf['pfaf_id'].astype(int)
df_wsf.set_index(['pfaf_id', 'year', 'scen'], inplace = True)

# 3. Perform weighted aggregation
We can do this for any indicator. I've selected water stress, water depletion, and interannual variability

## Find No Datas
We identified which countries do not have enough PCR-GLOBWB data in Aqueduct 3.0 Mask new data using these. 

In [5]:
# Read in Aqueduct 3.0 Carto Data
df_3ad0 = pd.read_csv(aq3cartoPATH('country')) 
df_3ad1 = pd.read_csv(aq3cartoPATH('province')) 
df_3ad1c = df_3ad1.drop_duplicates(subset = ['gid_1', 'indicator_name', 'weight'], keep='first')

# Create lists of no data
no_data_0 = df_3ad0.loc[(df_3ad0.indicator_name == 'bws') & (df_3ad0.weight == 'Tot') & (df_3ad0.score == -9999), 'gid_0'].tolist()
no_data_1 = df_3ad1c.loc[(df_3ad1c.indicator_name == 'bws') & (df_3ad1c.weight == 'Tot') & (df_3ad1c.score == -9999), 'gid_1'].tolist()

print('Countries: {}'.format(len(no_data_0)))
print('Provinces: {}'.format(len(no_data_1)))

Countries: 25
Provinces: 280


## Aggregate by admin unit

In [6]:
def perform_aggregation(gid, selected_indicators, no_data):
    print(gid)
    # 1. Sum withdrawal by country and state
    df_tot = df_wwm.groupby([gid, 'year', 'scen', 'sector'])[['value']].sum()
    df_tot.columns = ['tot']
    df_tot['tot_w'] = df_tot['tot']
    # 2. Create weights. Merge basin-state withdrawal water stress data with administrative total withdrawal
    df_weight = pd.merge(df_wwm, df_wsf, how = 'inner', left_on = ['pfaf_id', 'year', 'scen'], right_index = True)
    df_weight = pd.merge(df_weight, df_tot, how = 'left', left_on = [gid, 'year', 'scen', 'sector'], right_index = True)
    # 3. Find % of geometry withdrawal to total of country or state
    df_weight['wght'] = df_weight['value'].divide(df_weight['tot'])
    # 4. Multiple each indicator by the %
    for ind in selected_indicators:
        print("create weight for:", ind)
        df_weight[ind + '_w'] = df_weight[ind].multiply(df_weight['wght'])

    # 5. Finalize weighted sum for country and province
    weight_cols = [x + "_w" for x in selected_indicators] + ['tot_w']
    df_wgt = df_weight.groupby([gid, 'year', 'scen', 'sector'])[weight_cols].sum().reset_index()
    # Take a non-weighted average
    one_cols = selected_indicators + ['tot']
    df_one = df_weight.groupby([gid, 'year', 'scen', 'sector'])[one_cols].mean().reset_index()

    # 6. Now, find the median per SSP for future years
    # -- First, define SSP
    df_wgt['ssp'] = df_wgt['scen'].apply(lambda x: x.split("_")[1])
    df_one['ssp'] = df_one['scen'].apply(lambda x: x.split("_")[1])
    # # -- Second, find median
    df_wgt_med = df_wgt.groupby([gid, 'year', 'ssp', 'sector'])[weight_cols].median().reset_index()
    df_one_med = df_one.groupby([gid, 'year', 'ssp', 'sector'])[one_cols].median()
    df_one_med = df_one_med.add_suffix("_w").reset_index()

    # 7. Clean the data frames
    # -- For the non-weighted data, only keep the average from the total sum
    df_one_med = df_one_med.loc[df_one_med['sector'] == 'gtotww']

    # --Turn Sector column into weight column for both dataframes
    df_wgt_med['weight'] = df_wgt_med['sector'].apply(lambda x: x[1:4].title())
    df_one_med['weight'] = 'One'

    # 8. Merge weighted with non-weighted
    df_avgs = pd.concat([df_wgt_med, df_one_med], axis = 0)
    
    # Drop data set as no data if no data in Aqueduct 3.0
    df_avg_valid = df_avgs.loc[~df_avgs[gid].isin(no_data)]
    # 9. Find ranking
    # -- First, create blank rank column for each indicator
    for ind in selected_indicators:
        print("create rank for:", ind)
        df_avg_valid[ind + '_rank'] = np.nan
    # --Next, loop through each indicator, sector, year, and ssp and define rank for each
    for ind in selected_indicators:
#         print(i) 
        for w in ['Tot', 'Dom', 'Ind', 'Liv', 'Irr', 'One']:
#             print(w)
            for y in [2019, 2030, 2050, 2080]:
                for s in ['hist', 'ssp126', 'ssp370', 'ssp585']:
                    if y == 2019 and s != 'hist':
                        continue
                    elif y != 2019 and s == 'hist':
                        continue
#                     print(y, s)
                    df_avg_valid.loc[(df_avg_valid['weight'] == w) 
                                & (df_avg_valid['year'] == y)
                                & (df_avg_valid['ssp'] == s), ind + '_rank'] = df_avg_valid.loc[(df_avg_valid['weight'] == w) 
                                                                                 & (df_avg_valid['year'] == y)
                                                                                 & (df_avg_valid['ssp'] == s), ind + '_w'].rank(ascending = False)
    # Set index
    df_avg_valid.set_index([gid, 'year', 'ssp', 'weight'], inplace = True)
    return df_avg_valid

In [7]:
df_0 = perform_aggregation(gid = 'gid_0', selected_indicators = indicators, no_data = no_data_0)
df_1 = perform_aggregation(gid = 'gid_1', selected_indicators = indicators, no_data = no_data_1)

gid_0
create weight for: ws_s
create weight for: wd_s
create weight for: iv_s
create rank for: ws_s


C:\Users\Samantha.Kuzma\AppData\Local\Temp\ipykernel_23780\3466462613.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_avg_valid[ind + '_rank'] = np.nan
C:\Users\Samantha.Kuzma\AppData\Local\Temp\ipykernel_23780\3466462613.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_avg_valid[ind + '_rank'] = np.nan
C:\Users\Samantha.Kuzma\AppData\Local\Temp\ipykernel_23780\3466462613.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

create rank for: wd_s
create rank for: iv_s
gid_1
create weight for: ws_s
create weight for: wd_s
create weight for: iv_s
create rank for: ws_s
create rank for: wd_s
create rank for: iv_s


C:\Users\Samantha.Kuzma\AppData\Local\Temp\ipykernel_23780\3466462613.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_avg_valid[ind + '_rank'] = np.nan
C:\Users\Samantha.Kuzma\AppData\Local\Temp\ipykernel_23780\3466462613.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_avg_valid[ind + '_rank'] = np.nan
C:\Users\Samantha.Kuzma\AppData\Local\Temp\ipykernel_23780\3466462613.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [8]:
df_0.to_csv(workingPATH('countries'))
df_1.to_csv(workingPATH('provinces'))

# 4. Create final download and Carto table
We will use baseline water stress, drought risk and riverine flood risk in the final public document. Drought risk and riverine flood risk can be taken from Aqueduct 3.0 because we did not make any changes to the SCORE values (the drough RAW values were fixed. In Aq 3.0, they equaled the score by mistake)

In [39]:
ssp_dict = {'ssp126': 'opt', 'ssp370': 'bau', 'ssp585': 'pes'}

def clean_indicator(df_in, i, labels, gid):
    # filter data
    df_f = df_in.filter([i + "_w", i + "_rank"] + ['tot_w'])
    # rename indicator score, rank, and sum of weights
    df_f.columns = ['score', 'score_ranked', 'sum_weights']
    # calculate sum weight per indicator
    df_f['sum_weighted_indicator'] = df_f['sum_weights'].multiply(df_f['score'])
    # add clean indicator name
    df_f['indicator_name'] = ind_names.get(i)
    # add category and label
    df_f['cat'] = category(score = 'score', df_in = df_f)['cat']
    df_f['label'] = df_f['cat'].map(labels)
    # redo Index
    df_f.reset_index(inplace = True)
    df_f['scenario'] = df_f['ssp'].apply(lambda x: ssp_dict.get(x))
    df_f.drop(['ssp'], axis = 1, inplace = True)
    df_f.rename(columns = {'year':'period'}, inplace = True)
    df_f.set_index([gid, 'period', 'scenario', 'indicator_name', 'weight'], inplace = True)
    return df_f

## BASELINE

### COUNTRY

In [40]:
# Read in 3.0 country data. Create list of accepted countries
# IF USING DOWNLOADABLE DATA AS STARTING PLACE
# df_3ad0 = pd.read_excel(aq3PATH, sheet_name = 'results country')
# df_3ad0.rename(columns = {'iso_a3': 'gid_0'}, inplace = True)
# df_3ad0.set_index(['gid_0', 'indicator_name', 'weight'], inplace = True)

# IF USING CARTO AS STARTING PLACE
df_3ad0 = pd.read_csv(aq3cartoPATH('country'), index_col = ['gid_0', 'indicator_name', 'weight']) 
print('length of OG:', len(df_3ad0))

# Multiply old Drought score by 5
print('Original Max Drought Score: {}'.format(df_3ad0.loc[df_3ad0.index.get_level_values('indicator_name') == 'drr', 'score'].max()))
df_3ad0.loc[df_3ad0.index.get_level_values('indicator_name') == 'drr', 'score'] = df_3ad0.loc[df_3ad0.index.get_level_values('indicator_name') == 'drr', 'score'] * 5
# Reset -9999 * 5 to -9999
df_3ad0['score'] = df_3ad0['score'].mask(df_3ad0['score'] == -49995, -9999)
print('New Max Drought Score: {}'.format(df_3ad0.loc[df_3ad0.index.get_level_values('indicator_name') == 'drr', 'score'].max()))

# # Set BWS data to NaN 
data_cols = ['score', 'score_ranked', 'cat', 'label', 'sum_weights', 'sum_weighted_indicator',]
df_3ad0.loc[df_3ad0.index.get_level_values('indicator_name') == 'bws', data_cols] = np.nan

# # # Create clean version of baseline water stress
df_0ws = clean_indicator(df_in = df_0, i = 'ws_s', labels = ws_labels, gid = 'gid_0')
# # # Create a version of the baseline only
df_0wsb = df_0ws.loc[df_0ws.index.get_level_values('period') == 2019]
df_0wsb = df_0wsb.droplevel(['period', 'scenario'])
# # FIll WS data with new results
df_4ad0 = df_3ad0.fillna(df_0wsb)

# Drop old columns
df_4ad0.drop(['count_valid', 'fraction_valid', 'primary', 'valid_hybas6'], axis = 1, inplace = True)
# Set no data = -9999
df_4ad0.loc[(df_4ad0.index.get_level_values('indicator_name') == 'bws') & (df_4ad0['score'].isna()), data_cols] = -9999
df_4ad0.loc[(df_4ad0.index.get_level_values('indicator_name') == 'bws') & (df_4ad0['score'] == -9999), 'label'] = 'NoData'
print('length of new:', len(df_4ad0))

length of OG: 2456
Original Max Drought Score: 0.831943067
New Max Drought Score: 4.159715335
length of new: 2456


### PROVINCE

In [41]:
# Read in 3.0 country data. Create list of accepted countries
# IF USING DOWNLOADABLE DATA AS STARTING PLACE
# df_3ad1 = pd.read_excel(aq3PATH, sheet_name = 'results province')
# df_3ad1.rename(columns = {'iso_a3': 'gid_0'}, inplace = True)
# df_3ad1c = df_3ad1.drop_duplicates(subset = ['gid_1', 'indicator_name', 'weight'], keep='first')
# df_3ad1c.set_index(['gid_1', 'indicator_name', 'weight'], inplace = True)

# IF USING CARTO AS STARTING PLACE
df_3ad1 = pd.read_csv(aq3cartoPATH('province')) 
df_3ad1c = df_3ad1.drop_duplicates(subset = ['gid_1', 'indicator_name', 'weight'], keep='first')
df_3ad1c.set_index(['gid_1', 'indicator_name', 'weight'], inplace = True)
print('length of OG:', len(df_3ad1c))

# Multiply old Drought score by 5
print('Original Max Drought Score: {}'.format(df_3ad1c.loc[df_3ad1c.index.get_level_values('indicator_name') == 'drr', 'score'].max()))
df_3ad1c.loc[df_3ad1c.index.get_level_values('indicator_name') == 'drr', 'score'] = df_3ad1c.loc[df_3ad1c.index.get_level_values('indicator_name') == 'drr', 'score'] * 5
# Reset -9999 * 5 to -9999
df_3ad1c['score'] = df_3ad1c['score'].mask(df_3ad1c['score'] == -49995, -9999)
print('New Max Drought Score: {}'.format(df_3ad1c.loc[df_3ad1c.index.get_level_values('indicator_name') == 'drr', 'score'].max()))


# # Set BWS data to NaN 
data_cols = ['score', 'score_ranked', 'cat', 'label', 'sum_weights', 'sum_weighted_indicator',]
df_3ad1c.loc[df_3ad1c.index.get_level_values('indicator_name') == 'bws', data_cols] = np.nan

# # # Create clean version of baseline water stress
df_1ws = clean_indicator(df_in = df_1, i = 'ws_s', labels = ws_labels, gid = 'gid_1')
# # # Create a version of the baseline only
df_1wsb = df_1ws.loc[(df_1ws.index.get_level_values('period') == 2019)]
df_1wsb = df_1wsb.droplevel(['period', 'scenario'])
# # FIll WS data with new results
df_4ad1 = df_3ad1c.fillna(df_1wsb)

# Drop old columns
drop_cols = ['count_valid', 'fraction_valid', 'primary', 'valid_hybas6']
df_4ad1.drop(drop_cols, axis = 1, inplace = True)
# Set no data = -9999
df_4ad1.loc[(df_4ad1.index.get_level_values('indicator_name') == 'bws') & (df_4ad1['score'].isna()), data_cols] = -9999
df_4ad1.loc[(df_4ad1.index.get_level_values('indicator_name') == 'bws') & (df_4ad1['score'] == -9999), 'label'] = 'NoData'
print('length of new:', len(df_4ad1))

length of OG: 42771
Original Max Drought Score: 0.970316846
New Max Drought Score: 4.85158423


C:\Users\Samantha.Kuzma\AppData\Local\Temp\ipykernel_23780\2016594663.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3ad1c['score'] = df_3ad1c['score'].mask(df_3ad1c['score'] == -49995, -9999)


length of new: 42771


## FUTURE

### COUNTRY

In [42]:
# Clean water stress data
df_0ws = clean_indicator(df_in = df_0, i = 'ws_s', labels = ws_labels, gid = 'gid_0')
# # # Create a version of the future only
df_0wsf = df_0ws.loc[df_0ws.index.get_level_values('period') != 2019]
df_0wsf = df_0wsf.reset_index().set_index(['gid_0'])
# Read in 3.0 country data. Create list of accepted countries
df_3ad0 = pd.read_excel(aq3PATH, sheet_name = 'results country')
df_3ad0.rename(columns = {'iso_a3': 'gid_0'}, inplace = True)
# Create list of approved names from Aq 3.0 data
df_names = df_3ad0.loc[(df_3ad0['indicator_name'] == 'bws') & (df_3ad0['weight'] == 'Tot')]
df_names.set_index(['gid_0'], inplace = True)
df_names = df_names.filter(['iso_n3', 'name_0', 'un_region', 'wb_region'])
# Add name columns to future results
df_4ad0f = pd.merge(df_0wsf, df_names, how = 'inner', left_index = True, right_index = True)
df_4ad0f.reset_index().set_index(['gid_0', 'scenario', 'period', 'indicator_name', 'weight'], inplace = True)

### PROVINCE

In [43]:
# Clean water stress data
df_1ws = clean_indicator(df_in = df_1, i = 'ws_s', labels = ws_labels, gid = 'gid_1')
# # # Create a version of the baseline only
df_1wsf = df_1ws.loc[df_1ws.index.get_level_values('period') != 2019]
df_1wsf = df_1wsf.reset_index().set_index(['gid_1'])
# Read in 3.0 country data. Create list of accepted countries
df_3ad1 = pd.read_excel(aq3PATH, sheet_name = 'results province')
df_3ad1.rename(columns = {'iso_a3': 'gid_0'}, inplace = True)
df_names = df_3ad1.loc[(df_3ad1['indicator_name'] == 'bws') & (df_3ad1['weight'] == 'Tot')]
df_names.set_index(['gid_1'], inplace = True)
df_names = df_names.filter(['iso_n3', 'gid_0', 'name_0', 'name_1', 'un_region', 'wb_region'])
df_4ad1f = pd.merge(df_1wsf, df_names, how = 'inner', left_index = True, right_index = True)
df_4ad1f.reset_index().set_index(['gid_1', 'scenario', 'period', 'indicator_name', 'weight'], inplace = True)

# SAVE

In [44]:
df_4ad0.to_csv(cartoPATH('country'))
df_4ad1.to_csv(cartoPATH('province'))
df_4ad0f.to_csv(cartoPATH('country_future'))
df_4ad1f.to_csv(cartoPATH('province_future'))